In [16]:
import import_ipynb
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2

output_path = "/content/drive/MyDrive/Waste Classification Project/dataset_processed"

# import dataset_prep # Import dataset preprocessing script

# train_generator = dataset_prep.train_generator
# val_generator = dataset_prep.val_generator
# test_generator = dataset_prep.test_generator

In [ ]:
# Set up ImageDataGenerator for augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to between 0 and 1
    rotation_range=40,         # Random rotation range in degrees
    width_shift_range=0.2,     # Horizontal shift
    height_shift_range=0.2,    # Vertical shift
    shear_range=0.2,           # Shear transformation
    zoom_range=0.2,            # Zoom in/out
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill missing pixels after transformations
)

# Apply preprocessing and augmentation on the training set
train_generator = datagen.flow_from_directory(
    os.path.join(output_path, 'train'),
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='binary'      # Binary classification: hazardous vs non-hazardous
)

# Set up a simple validation generator (no augmentation, only rescaling)
test_val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = test_val_datagen.flow_from_directory(
    os.path.join(output_path, 'val'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Test generator (no augmentation)
test_generator = test_val_datagen.flow_from_directory(
    os.path.join(output_path, 'test'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

print("Dataset preprocessing, augmentation, and splitting completed!")

In [ ]:
# Load pre-trained MobilenetV2 as base model
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model layers

# Define the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') # Binary classification (hazardous vs non-hazardous)
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Model summary
model.summary()

In [19]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 1812s 11s/step - accuracy: 0.8051 - loss: 2.8608 - val_accuracy: 0.9638 - val_loss: 0.1020
Epoch 2/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 430s 3s/step - accuracy: 0.9280 - loss: 0.1935 - val_accuracy: 0.9638 - val_loss: 0.0833
Epoch 3/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 429s 3s/step - accuracy: 0.9281 - loss: 0.1887 - val_accuracy: 0.9683 - val_loss: 0.0833
Epoch 4/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 420s 3s/step - accuracy: 0.9310 - loss: 0.1663 - val_accuracy: 0.9701 - val_loss: 0.0702
Epoch 5/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 450s 3s/step - accuracy: 0.9455 - loss: 0.1519 - val_accuracy: 0.9611 - val_loss: 0.1117
Epoch 6/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 441s 3s/step - accuracy: 0.9386 - loss: 0.1763 - val_accuracy: 0.9747 - val_loss: 0.0757
Epoch 7/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 424s 3s/step - accuracy: 0.9466 - loss: 0.1422 - val_accuracy: 0.9701 - val_loss: 0.0829
Epoch 8/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 443s 3s/step - accuracy: 0.9507 - loss: 0.1259 - val_ac

In [21]:
# Save the trained model to Google Drive
model.save('/content/drive/MyDrive/Waste Classification Project/model/Model.h5')

print("Model saved successfully!")

Model saved successfully!


In [20]:
# Evaluate model on test dataset
test_loss, test_acc = model.evaluate(test_generator)

print(f"Test Accuracy: {test_acc * 100:.2f}%")

35/35 ━━━━━━━━━━━━━━━━━━━━ 482s 14s/step - accuracy: 0.9862 - loss: 0.0499
Test Accuracy: 98.39%
